## Finding Missing Requirements

In this notebook, I try to put together an iterative way to:
1. Add *unbound variables* until the requirement is satisfied
2. Get the *relationships* of the unbound variables
3. Filter these relationships by adding one at a time as a negated constraint
   and check again the model.
4. When we find the one that makes the model unsat, it means we've found the right one.

#### Goals
- Make the search of finding the correct relationship as fast as possible:
  usually relationships are quite a lot, and we need to iterate them at least a
  couple of times. We can add progressively only the relationships that remain
  after each pass, always one at a time and checking the model again.

In [32]:
from mc_openapi.doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_doml_model
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_xmi_model
from mc_openapi.doml_mc import DOMLVersion
from z3 import Solver, Model, DatatypeSortRef, FuncDeclRef, Consts, Const, ForAll, Exists, Implies, And, Or, Not, Bools, sat, unsat
import yaml

from mc_openapi.doml_mc.z3encoding.metamodel_encoding import (
    def_association_rel,
    def_attribute_rel,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from mc_openapi.doml_mc.z3encoding.im_encoding import (
    assert_im_associations,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict,
    mk_attr_data_sort
)
from mc_openapi.doml_mc.z3encoding.types import Refs

from itertools import product
from operator import itemgetter

In [33]:
with open("../assets/doml_meta_v2.0.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

**You can change here the input DOML file**

In [34]:
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0.domlx"
doml_document_path = "../../tests/doml/v2.0/nginx-openstack_v2.0_wrong_vm_iface.domlx"
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0_wrong_iface_uniq.domlx"


In [35]:
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

im, _ = parse_doml_model(doc, DOMLVersion.V2_0)
doml_xmi = parse_xmi_model(doc, DOMLVersion.V2_0)

We need to initialize each time the Solver context before iterating,
since an unbound variable is an element, and elements are an EnumSort,
and EnumSorts cannot be modified and depend on the solver context.

The following code is stuff that is already present in the `IntermediateModelChecker`.

In [36]:
from typing import Dict

Context = Dict

In [37]:
def initialize_solver(
    unbound_elems_quantity: int = 0,
    requirements: list = []
) -> Context:
    ctx = dict()
    
    ctx["solver"] = Solver()

    ctx["class_sort"], ctx["class_"] = mk_class_sort_dict(mm, ctx["solver"].ctx)
    ctx["assoc_sort"], ctx["assoc"] = mk_association_sort_dict(mm, ctx["solver"].ctx)
    ctx["attr_sort"], ctx["attr"] = mk_attribute_sort_dict(mm, ctx["solver"].ctx)
    ctx["str_sort"], ctx["str"] = mk_stringsym_sort_dict(im, mm, ctx["solver"].ctx)
    ctx["attr_data_sort"] = mk_attr_data_sort(ctx["str_sort"], ctx["solver"].ctx)

    ctx["unbound_elems"] = [f"unbound{i}" for i in range(unbound_elems_quantity)]
    ctx["elem_sort"], ctx["elem"] = mk_elem_sort_dict(im, ctx["solver"].ctx, ctx["unbound_elems"])

    ctx["elem_class_f"] = def_elem_class_f_and_assert_classes(
        im,
        ctx["solver"],
        ctx["elem_sort"],
        ctx["elem"],
        ctx["class_sort"],
        ctx["class_"]
    )
    # attr_rel :: (elem_sort, attr_sort, attr_data_sort) -> BoolRef
    ctx["attr_rel"] = def_attribute_rel(
        ctx["attr_sort"],
        ctx["elem_sort"],
        ctx["attr_data_sort"]
    )
    assert_im_attributes(
        ctx["attr_rel"],
        ctx["solver"],
        im,
        mm,
        ctx["elem"],
        ctx["attr_sort"],
        ctx["attr"],
        ctx["attr_data_sort"],
        ctx["str"]
    )

    # assoc_rel :: (elem_sort, assoc_sort, elem_sort) -> BoolRef
    ctx["assoc_rel"] = def_association_rel(
        ctx["assoc_sort"],
        ctx["elem_sort"]
    )
    assert_im_associations(
        ctx["assoc_rel"],
        ctx["solver"],
        {k: v for k, v in im.items() if k not in ctx["unbound_elems"]},
        ctx["elem"],
        ctx["assoc_sort"],
        ctx["assoc"],
    )

    # Add requirements
    for req in requirements:
        req(ctx)

    return ctx

In [38]:
def req_every_vm_has_iface(ctx: Context):    
    vm, iface = Consts("vm iface", ctx["elem_sort"])
    vmIfaceAssertion = ForAll(
        [vm],
        Implies(
            ctx["elem_class_f"](vm) == ctx["class_"]["infrastructure_VirtualMachine"],
            Exists(
                [iface],
                And(
                    ctx["assoc_rel"](vm, ctx["assoc"]["infrastructure_ComputingNode::ifaces"], iface)
                )
            )
        )
    )
    ctx["solver"].assert_and_track(vmIfaceAssertion, "vm_iface")

In [39]:
def req_every_iface_has_a_secgroup(ctx):
    sg, iface = Consts("sg iface", ctx["elem_sort"])
    vmIfaceSecGroupAssertion = ForAll(
        [sg],
        Implies(
            ctx["elem_class_f"](sg) == ctx["class_"]["infrastructure_SecurityGroup"],
            Exists([iface], 
                ctx["assoc_rel"](iface, ctx["assoc"]["infrastructure_NetworkInterface::associated"], sg)
            )
        )
    )
    ctx["solver"].assert_and_track(vmIfaceSecGroupAssertion, "vm_secgroup")

In [40]:
# There are no duplicated interfaces.
def req_iface_uniq(ctx):
    endPointAttr = ctx["attr"]["infrastructure_NetworkInterface::endPoint"]
    ni1, ni2 = Consts("ni1, ni2", ctx["elem_sort"])
    value = Const("value", ctx["attr_data_sort"])
    uniqueIfaceAssertion = And(
        ctx["attr_rel"](ni1, endPointAttr, value),
        ctx["attr_rel"](ni2, endPointAttr, value),
        ni1 != ni2,
    )
    ctx["solver"].assert_and_track(uniqueIfaceAssertion, "unique_iface")

In [41]:
def check_with_ubvars(ubvars_n: int = 0, requirements: list = []) -> Context:
    ctx = initialize_solver(ubvars_n, requirements)
    solver = ctx["solver"]

    res = solver.check()

    if res == sat:
        print(f"Sat with {ubvars_n} unbounded variables")
        return ctx
    elif res == unsat:
        # print(f"ubvars={ubvars_n}; UNSAT_CORE:")
        # print(solver.unsat_core())
        print(f"Increasing unbound vars to {ubvars_n + 1}")
        return check_with_ubvars(ubvars_n + 1, requirements=requirements)
    else:
        raise RuntimeError("It took too long to decide.")

In [42]:
REQUIREMENTS = [
    req_every_vm_has_iface,
    req_every_iface_has_a_secgroup,
    req_iface_uniq
]

In [43]:
solved_ctx = check_with_ubvars(requirements=REQUIREMENTS)
solved_model = solved_ctx["solver"].model()

Increasing unbound vars to 1
Increasing unbound vars to 2
Sat with 2 unbounded variables


In [44]:
def get_ubvars_and_assoc(ctx: Context, model: Model):
    elem, assoc, assoc_rel, unbound_elems = itemgetter("elem", "assoc", "assoc_rel", "unbound_elems")(ctx)

    return [ ((e1n, e1), a, (e2n, e2)) 
        for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()) 
        if (e1n in unbound_elems or e2n in unbound_elems) and model.eval(assoc_rel(e1, a, e2))
    ]

In [45]:
def pretty_ubvar_assoc(assoc):
    (e1n, e1), a, (e2n, e2) = assoc
    tokens = str(str(e1) + " " + str(a) + " " + str(e2)).split()
    ret_str = ""
    for token in tokens:
        if token[0:4] == "elem":
            value = im.get(token)
            ret_str += f"{value.class_} ({value.user_friendly_name})" if value else f"<'{token}' not found>"
        else:
            ret_str += token
        ret_str += " "
    return ret_str.strip()

As we can see, there are many associations that involve the unbound variable `unbound0`.
We can see that between them, there is the one we really want:

`infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0`

We now need to filter out all the others, by taking one of these lines at a time, and adding it as a negated constraint.

NOTE: It appears that the list is not deterministic. Sometimes at the end there are associations that start from `unbound0`, sometimes there is none. 

In [46]:
ubvars_and_assoc = get_ubvars_and_assoc(solved_ctx, solved_model)
# print("\n".join([pretty_ubvar_assoc(assoc) for assoc in ubvars_and_assoc]))

In [52]:
def thin_ubvars_and_assoc(ctx: Context, ubvars_and_assoc: list):
    """Returns a tuple where the first item is
    """
    if not ubvars_and_assoc:
        return []


    (e1_name, e1), a, (e2_name, e2) = assoc = ubvars_and_assoc[0]
    assoc_rel = ctx["assoc_rel"](e1, a, e2)
    
    print("-----------------------------------------")

    # Add negated constraint
    print(f"\tAdd constraint Not({pretty_ubvar_assoc(ubvars_and_assoc[0])})")
    ctx["solver"].push()
    # ctx["solver"].assert_and_track(Not(assoc_rel), f"__neg_{e1_name}_{str(a)}_{e2_name}")
    ctx["solver"].add(Not(assoc_rel))
    
    res = ctx["solver"].check()
    if res == sat:
        print("SAT:\tAdding one more constraint and trying again")
        # Get new ubvars_and_assoc
        model = ctx["solver"].model()
        thinned_ubvars_and_assoc = get_ubvars_and_assoc(ctx, model)
        
        # Print table showing the diff
        from difflib import context_diff
        uvar_as_text = lambda input: [pretty_ubvar_assoc(assoc) for assoc in input]
        print("\n".join([a for a in context_diff(uvar_as_text(ubvars_and_assoc), uvar_as_text(thinned_ubvars_and_assoc), lineterm="", fromfile='Before', tofile="After")]))

        # Iterate
        return thin_ubvars_and_assoc(ctx, thinned_ubvars_and_assoc)
    else:
        print("UNSAT\tLast constraint was the association we are looking for!")
        ctx["solver"].pop()
        
        if ubvars_and_assoc[1:]:
            print("\tIterating over")
            print("\t\t" + "\n\t\t".join([pretty_ubvar_assoc(assoc) for assoc in ubvars_and_assoc[1:]]))
        return [*set([assoc] + thin_ubvars_and_assoc(ctx, ubvars_and_assoc[1:]))]


assoc_to_implement = thin_ubvars_and_assoc(solved_ctx, ubvars_and_assoc)

-----------------------------------------
	Add constraint Not(infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound0)


Z3Exception: True, False or Z3 Boolean expression expected. Received [__neg_{e1_name}_{str(a)}_{e2_name}] of type <class 'list'>

In [ ]:
print("\n".join([pretty_ubvar_assoc(assoc) for assoc in assoc_to_implement]))

infrastructure_VirtualMachine (vm1) infrastructure_ComputingNode::ifaces unbound1
unbound1 infrastructure_NetworkInterface::associated infrastructure_SecurityGroup (sg)


From here, we should then add this in the Intermediate Model, and then in the ECore (?) to generate the DOML file somehow.

We can then provide the new file, or a diff to be patched onto the original?